<a href="https://colab.research.google.com/github/Taniisha/Face-Mask-Detection/blob/master/Face_mask_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# setting the environment


In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145480 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.23-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Preprocessing**

In [ ]:
# changing the directory to th required directory to access the dataset

import os

os.chdir("/content/drive/MyDrive/Colab Notebooks/OpenCV")
Dataset = "Dataset"
Data_Dir = os.listdir(Dataset)    #os. listdir() method in python is used to get the list of all files and directories in the specified directory.
print(Data_Dir)


['with_mask', 'without_mask']


In [ ]:
import cv2 
import numpy as np
from tensorflow.keras.utils import to_categorical   #Using the method to_categorical() , a numpy array (or) a vector which has integers that represent different categories, can be converted into a numpy array (or) a matrix which has binary values and has columns equal to the number of categories in the data.
from sklearn.preprocessing import LabelBinarizer    #It assigns a unique v alue or number to each label in a categorical feature. 
from sklearn.model_selection import train_test_split   #to divide the dataset into train and test


In [ ]:
#dimensions of image

img_rows , img_cols = 112,112
images = []         #it will store all the final images(after gray scaling and resizing)
labels = []         #it will contain labels corresponding to classes of image(withMask , withoutMask)

for category in Data_Dir:
  folder_path = os.path.join(Dataset , category)    #we are joining dataset folders with_mask and without_mask to access them
  for img in os.listdir(folder_path):
    img_path = os.path.join(folder_path,img)
    img  = cv2.imread(img_path)

    try:
      #converting img into grayscale image
      grayscale_img =  cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

      #resizing the img 
      resized_img = cv2.resize(grayscale_img , (img_rows, img_cols))
      images.append(resized_img)
      labels.append(category)

    except Exception as e:
      print("Exception : ",e)
    
    


In [ ]:
images = np.array(images)/255.0          #to have pixel values in the range 0-1, dividing all pixels values by the largest pixel value; that is 255. This is performed across all channels, regardless of the actual range of pixel values that are present in the image
images = np.reshape(images , (images.shape[0], img_rows , img_cols,1))      #converting into 3Dmatrix
                                                                   # 1 representing gray scale                                                                   

In [ ]:
#performing one hot encoding on the labels (with_mask , without_mask) because we have string data
lb = LabelBinarizer()
labels = lb.fit_transform(labels)     #fitting and transforming data
labels = to_categorical(labels)       #converting to categorical
labels = np.array(labels)             #converting into array because our models understand data in the form of vector or array

(train_X,test_X,train_y,test_y) = train_test_split(images,labels,test_size=0.25,random_state=0)


# **CNN model** 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout,Activation
from keras.layers import Conv2D,MaxPooling2D

In [ ]:
num_classes = 2   #since our output is binary(with_mask and without_mask)
batch_size = 32


In [ ]:
model = Sequential()

model.add(Conv2D(64,(3,3),input_shape = (img_rows,img_cols,1),activation='relu'))
#model.add(Activation='relu')
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),activation='relu'))
#model.add(Activation='relu')
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())    #Flatten layer converts data into 1D array so that it can be fed to other layer
model.add(Dropout(0.5))   #it is a regularization parameter to reduce overfitting

model.add(Dense(64,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 110, 110, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 53, 53, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 86528)             0         
_________________________________________________________________
dropout (Dropout)            (None, 86528)             0         
_________________________________________________________________
dense (Dense)                (None, 64)               

# **Train Model**

In [ ]:
from keras.optimizers import Adam

epochs = 55

model.compile(optimizer = Adam(lr=0.001), loss= 'categorical_crossentropy' , metrics=['accuracy'])

fit_model = model.fit(train_X,train_y,epochs=epochs,validation_split=0.25)



Epoch 1/55
14/14 [==============================] - 1s 40ms/step - loss: 0.0052 - accuracy: 0.9994 - val_loss: 0.3020 - val_accuracy: 0.9320
Epoch 2/55
14/14 [==============================] - 0s 26ms/step - loss: 0.0068 - accuracy: 0.9990 - val_loss: 0.4856 - val_accuracy: 0.9048
Epoch 3/55
14/14 [==============================] - 0s 26ms/step - loss: 0.0171 - accuracy: 0.9931 - val_loss: 0.3973 - val_accuracy: 0.9184
Epoch 4/55
14/14 [==============================] - 0s 26ms/step - loss: 0.0198 - accuracy: 0.9937 - val_loss: 0.3619 - val_accuracy: 0.9116
Epoch 5/55
14/14 [==============================] - 0s 26ms/step - loss: 0.0053 - accuracy: 0.9985 - val_loss: 0.4095 - val_accuracy: 0.9320
Epoch 6/55
14/14 [==============================] - 0s 26ms/step - loss: 0.0084 - accuracy: 0.9990 - val_loss: 0.3533 - val_accuracy: 0.9184
Epoch 7/55
14/14 [==============================] - 0s 26ms/step - loss: 0.0022 - accuracy: 0.9997 - val_loss: 0.3057 - val_accuracy: 0.9252
Epoch 8/55
14

# **Save or Serializ the model**

In [ ]:
model.save('mask_detection.h5')